In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import re
import mysql.connector
import pymysql
import os
from dotenv import load_dotenv

In [ ]:
class newsScrap :
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}
    
    def __init__(self, company):
        self.company = company
        self.titles = []
        self.dates = []
        self.articles = []
        self.article_urls = []
        self.press_companies = []
        
    def createNewsLinks(self):
        start_point = 1
        while True :
            #pd => 4=1일 , 2= 1달
            url = 'https://search.naver.com/search.naver?where=news&query='+ str(self.company) + '&sm=tab_opt&sort=1&photo=0&field=0&pd=4&start=' + str(start_point)
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')
            
            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    self.article_urls.append(urls["href"])
#                     print(urls["href"])
            is_last_page = source.find('a',{'class':'btn_next'}).get('aria-disabled')
            if is_last_page == "true":
#                 print("last page")
                break
            else:
                time.sleep(1)
                start_point += 10
                
                    
    def newsDataScrap(self):
        for url in self.article_urls:
            try:
                web_news = requests.get(url, headers=self.headers).content
                source_news = BeautifulSoup(web_news, 'html.parser')

                title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()

                date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                article = source_news.find('div', {'id' : 'newsct_article'}).get_text()
                article = article.replace("\n", "")
                article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                article = article.replace("동영상 뉴스       ", "")
                article = article.replace("동영상 뉴스", "")
                article = article.strip()

                press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                
            
                self.titles.append(title)
                self.dates.append(date)
                self.articles.append(article)
                self.press_companies.append(press_company)
            except:
                pass
            
            
            
    def saveExcel(self):
        article_df = pd.DataFrame({'Title':self.titles, 
                                   'Date':self.dates, 
                                   'Article':self.articles, 
                                   'URL':self.article_urls, 
                                   'PressCompany':self.press_companies})
        article_df.to_excel('{}_{}.xlsx'.format(self.company, datetime.now().strftime('%y%m%d_%H%M')), index=False)
        
        
    def dbUpdater(self):
        config = {'user': '',
                'password': '',
                'host': '',
                'database': '',
                'raise_on_warnings': True}
        
        cnn = pymysql.connect(host='', 
                              user='', password='',db='')
        querys_data = []
        
        cur = cnn.cursor()
        
        #테이블에 맞춰 항목 추가 예정 
        for i in range(len(self.article_urls)):
            data = (i, self.titles[i], self.articles[i], self.dates[i], self.press_companies[i],self.article_urls[i])
            querys_data.append(data)
        
        query = 'INSERT INTO news_data VALUES (%s, %s, %s, %s, %s, %s)'
        #query = "INSERT INTO test (name, City) VALUES (%s, %s)"
        
        for i in querys_data:
            cur.execute(query, i)
        
        cnn.commit()
        cur.close()
        cnn.close()

In [ ]:
test = newsScrap('롯데백화점')

In [ ]:
test.createNewsLinks()

In [ ]:
test.article_urls

In [ ]:
test.newsDataScrap()

In [ ]:
test.articles[0:2]

In [ ]:
test.dbUpdater()

In [ ]:
def divideArticle(artList):
    divideArt = []
    for art in artList:
        artTemp = art.split('.')
        divideArt.append(artTemp)
    return divideArt

In [ ]:
ditest = divideArticle(test.articles[0:2])

In [ ]:
config = {'user': '',
            'password': '',
            'host': '',
            'database': '',
            'raise_on_warnings': True}
cnn = pymysql.connect(host='', user='', password='',db='')        
cur = cnn.cursor()

query = 'SELECT idx, news_id, enter_id FROM news_data'
query2 = 'SELECT enter_id FROM enterprise_data WHERE enter_name = %s'
query3 = 'SELECT * FROM enterprise_data'
data = '한화'
cur.execute(query)


result = cur.fetchall()
# print(result)
# print(type(result))
# print(str(result))
# print(type(result))
# print(result[0][0])
for row in result:
     print(row)
cur.close()
cnn.close()

In [10]:
load_dotenv()
host=os.getenv("HOST")
user=os.getenv("USER_NAME")
password=os.getenv("PASSWORD")
database=os.getenv("DATABASE")
cnn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
cur = cnn.cursor()

query = 'SELECT idx, news_id, enter_id FROM news_data'
query2 = 'SELECT enter_id FROM enterprise_data WHERE enter_name = %s'
query3 = 'SELECT * FROM enterprise_data'
data = '한화'
cur.execute(query)


result = cur.fetchall()
# print(result)
# print(type(result))
# print(str(result))
# print(type(result))
# print(result[0][0])
for row in result:
     print(row)
cur.close()
cnn.close()

(1, '철도시설 유지보수 코레일 독점깨지나', 'ER01')
(2, '“3683만원 넘으면 주차 금지”…LH 임대아파트 공지 ‘갑론을박’', 'ER01')
(3, "국토부, '철도 유지보수 코레일 독점' 규정 개정 추진", 'ER01')
(4, '국회 산자중기위, 국감 불출석 이상필·양계호 고발', 'ER01')
(5, '경실련 “LH 용역 80%는 전관업체가 수주”', 'ER01')
(17, '홍익표 "조태용 돌려막기, 안보가 장난인가"…\'회전문\' 교체 지적', 'ER01')
(18, "이천시, '2023 대한민국주거복지대전' 국토부장관상 수상", 'ER01')
(19, '심상정 "박상우 후보자 사외이사 신탁회사서도 전세사기 발생"', 'ER01')
(20, 'LH 자문위원 사칭 \'200억 사기\' 서준혁 첫 재판…"현재 직업 없다"', 'ER01')
(21, '혼인·출산 장려 지원정책 봇물…내년에 달라지는 부동산 제도는[실전재테크]', 'ER01')
(22, '박상우 국토장관 청문회…與 "국토전문가" 野 "전관예우 부적절"', 'ER01')
(23, '민주, 尹정부 인사 맹비판 "청문회가 고통"', 'ER01')
(24, '홍익표 "조태용 돌려막기, 안보가 장난인가"…\'회전문\' 교체 지적', 'ER01')
(25, "이천시, '2023 대한민국주거복지대전' 국토부장관상 수상", 'ER01')
(26, '심상정 "박상우 후보자 사외이사 신탁회사서도 전세사기 발생"', 'ER01')
(27, '4년만에 만난 한·중 대표 기업인… 소재·부품 등 공급망 원활화 협력', 'CH02')
(28, '올해 방산수출 130억 달러 웃돌아…수출국·무기체계 다변화', 'CH02')
(29, '올해 국내 대기업 M&A 지난해보다 62% 감소', 'CH02')
(30, '[헤럴드광장] K-방산 대장주, 결국 수출이 생명줄', 'CH02')
(31, '한화임팩트·한화파워시스템, 100% 수소연료 터빈 가동 성공', 'CH02')
(32, '재계 CEO들을 대통령 행사에 

In [7]:
class newsScrap :
    
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}
    
    def __init__(self, company):
        self.company = company
        self.titles = []
        self.dates = []
        self.articles = []
        self.article_urls = []
        self.press_companies = []
        self.url_full = []
        
    def createNewsLinks(self):
        start_point = 1
        while True :
            #pd => 4=1일 , 2= 1달
            url = 'https://search.naver.com/search.naver?where=news&query='+ str(self.company) + '&sm=tab_opt&sort=1&photo=0&field=0&pd=4&start=' + str(start_point)
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')
            
            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    self.article_urls.append(urls["href"])
                    # print(urls["href"])
            is_last_page = source.find('a',{'class':'btn_next'}).get('aria-disabled')
            if is_last_page == "true":
                # print("last page")
                break
            else:
                time.sleep(1)
                start_point += 10
                
                    
    def newsDataScrap(self):
        for url in self.article_urls:
            try:
                web_news = requests.get(url, headers=self.headers).content
                source_news = BeautifulSoup(web_news, 'html.parser')

                title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()

                date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                article = source_news.find('div', {'id' : 'newsct_article'}).get_text()
                article = article.replace("\n", "")
                article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                article = article.replace("동영상 뉴스       ", "")
                article = article.replace("동영상 뉴스", "")
                article = article.strip()

                press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                
                
                self.titles.append(title)
                self.dates.append(date)
                self.articles.append(article)
                self.press_companies.append(press_company)
                self.url_full.append(url)
            except:
                pass
            
            
            
    def saveExcel(self):
        article_df = pd.DataFrame({'Title':self.titles, 
                                   'Date':self.dates, 
                                   'Article':self.articles, 
                                   'URL':self.url_full, 
                                   'PressCompany':self.press_companies})
        article_df.to_excel('{}_{}.xlsx'.format(self.company, datetime.now().strftime('%y%m%d_%H%M')), index=False)
        
        
    def dbUpdater(self):
        load_dotenv()

        host=os.getenv("HOST")
        user=os.getenv("USER_NAME")
        password=os.getenv("PASSWORD")
        database=os.getenv("DATABASE")

        cnn = pymysql.connect(host=host, user=user, password=password, database=database)
        
        querys_data = []
        
        cur = cnn.cursor()
        
        selectQuery = 'SELECT enter_id FROM enterprise_data WHERE enter_name = %s'
        
        cur.execute(selectQuery, self.company)
        
        e_id = cur.fetchall()
#         print(self.company)
#         print(type(self.company))
        
        #테이블에 맞춰 항목 추가 예정 
        for i in range(len(self.titles)):
            data = (self.titles[i], self.articles[i], e_id[0][0], self.url_full[i])
            querys_data.append(data)
        
        query = 'INSERT INTO news_data(news_id, news_doc, enter_id, url) VALUES (%s, %s, %s, %s)'
        #query = "INSERT INTO test (name, City) VALUES (%s, %s)"
        
        for i in querys_data:
            cur.execute(query, i)
        
        cnn.commit()
        cur.close()
        cnn.close()

In [8]:
test = newsScrap('쿠팡')
test.createNewsLinks()
# print(test.article_urls)
# print(len(test.article_urls))
test.newsDataScrap()
# print(test.titles)
# print(len(test.titles))
# print(len(test.url_full))

In [ ]:
test.saveExcel()

In [9]:
test.dbUpdater()